
# **HiFi-GAN Fine Tuning (Kaggle Notebook)**
---
<a href="https://github.com/jik876/hifi-gan"> HiFi-GAN </a> | **Created by ColdFir3#9543 (Michael)**

This notebook will require: **A tacotron 2 trained model(22050hz), the dataset (same dataset and transcription file used to make the model)**

The dataset should look like this: 

```
Kaggle Dataset/
          ├──wavs/
          │    ├──1.wav
          │    ├──2.wav
          │    ├──3.wav
          │    └──etc
          └──transcription.txt
```

**You can train a tacotron 2 model <a href="https://www.kaggle.com/coldfir4/tacotron-2-training-kaggle-notebook"> HERE </a>**

**MAKE SURE 'GPU' HAS BEEN SELECTED AS THE ACCELERATOR IN THE NOTEBOOK SETTINGS**

*UPDATED 15/10/21 VERSION 3: Added continuation for training the HiFi-GAN model*

*UPDATED 22/10/21 VERSION 4: Added the ability to change batch size IF NEEDED*

*UPDATED 22/10/21 VERSION 5: Fixed bugs*

*UPDATED 23/10/21 VERSION 6: Fixed bugs*

---
# TRAINING INSTRUCTIONS
* **Make sure to make your own version of this notebook for each new model**
* **Import your dataset and your tacotron 2 model (22050hz) in the top right corner of the screen**
* **RUN ALL**
* **Fill in required inputs**
* **Follow though all steps**
---
# CONTINUING TO TRAIN?
* If returning back to continue training your model **MAKE SURE TO RUN ALL AND ENTER REQUIRED DATA**

* Import your Model as a "dataset" **SEPERATELY** from the other datasets previously added

* The "dataset" should contain the name of your character with all files within them

* **DON'T MAKE ANY OF THE SETTINGS DIFFERENT FROM YOUR PREVIOUS TRAINING**

* Recommended to name your "input" for your AI Model to be the same name as the model itself **(WILL SAVE LOTS OF TROUBLE AT STEP 3.5)**

* ***(VERY IMPORTANT)*** Once your model has been trained **DO NOT FORGET TO SAVE VERSION USING 'QUICK SAVE' AS THE OPTION AND GO TO 'ADVANCED' AND CHECK 'ALWAYS SAVE OUTPUT'** so you do NOT LOSE progress
---

# 1 Apply Settings via user input

In [1]:
import json
import pandas as pd

# print("NOT THE MODEL FILENAME ITSELF, ONLY THE NAME YOU HAD GIVEN IT WHEN IMPORTING AS DATA INTO KAGGLE")
# dataset_model_name = input("What is the name of the name you had assigned your TACOTRON2 MODEL within your KAGGLE ACCOUNT?: ")
# model_name = input("What is the name of your Tacotron 2 model?: ")
# dataset_name = input("What is the name of your dataset?: ")
transcription_name = f'/kaggle/input/tacotron-own-voice-training/metadata.csv'
# CHARACTERNAME = input("What is the name of your character? (No spaces, use only '-' or '_' or '.'): ")
BATCH_SIZE_INP = 32
# while BATCH_SIZE_INP <= -1:
#     BATCH_SIZE_INP = int(input("What batch size would you like to use (recommended: 16): "))

# while " " in CHARACTERNAME:
#     CHARACTERNAME = input("What is the name of your character? (No spaces, use only '-' or '_' or '.'): ")
tacotron_model = f"/kaggle/input/tacotroncheckpoint/checkpoint_20"
tacotron_dataset = f"/kaggle/input/tacotron-own-voice-training/wav/wav"
# train_filelist = f"/kaggle/input/textfiles/train.txt"
# val_filelist = f"/kaggle/input/textfiles/validation.txt"
output_dir = f"/kaggle/working/hifimodels/-"
print("Settings saved")

Settings saved


---
# 2 Download HiFi-GAN and Tacotron 2

In [2]:
print("Downloading and Intalling Tacotron 2 and HiFi-GAN...")
!pip install tensorflow==1.15.2
import os
!git clone -q https://github.com/SortAnon/tacotron2.git
os.chdir('/kaggle/working/tacotron2')
!pip install inflect
!git submodule init
!git submodule update
!apt-get install pv
!apt-get install jq
!pip install gdown
os.chdir('/kaggle/working')
!git clone -q https://github.com/harshalplus1/hifi-gan.git
!pip install -q unidecode tensorboardX

     |████████████████████████████████| 110.5 MB 68.7 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 48.4 MB/s eta 0:00:01     |████▌                           | 532 kB 48.4 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 33.6 MB/s eta 0:00:01
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=6909fad8148622592d26d0cf060c2df34b28bc17a3dd7dfd76dbef005abfc461
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0

---
# 3 Assign batch size and generate ground truth-aligned spectrograms. This will help HiFi-GAN learn what your Tacotron model sounds like

In [3]:
!apt-get update
!apt-get install sox -y

Get:1 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [5004 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://packages.cloud.google.com/apt cloud-sdk-bionic InRelease [6396 B] 
Get:4 http://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]        
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Err:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
  Certificate verification failed: The certificate is NOT trusted. The certificate issuer is unknown.  Could not handshake: Error in the certificate verification. [IP: 152.195.19.142 443]
Err:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
  Certificate verification failed: The certificate is NOT trusted. The certificate issuer is unknown.  Could not handshake: Error in the certificate verification. [IP: 152.195.19.142 443]

In [4]:
!soxi /kaggle/input/tacotron-own-voice-training/wav/wav/000.wav



Input File     : '/kaggle/input/tacotron-own-voice-training/wav/wav/000.wav'
Channels       : 1
Sample Rate    : 32000
Precision      : 16-bit
Duration       : 00:00:01.54 = 49392 samples ~ 115.763 CDDA sectors
File Size      : 98.8k
Bit Rate       : 512k
Sample Encoding: 16-bit Signed Integer PCM



In [6]:
cd tacotron2

/kaggle/working/tacotron2


In [7]:
mkdir wav

In [8]:
import os
for file in os.listdir('/kaggle/input/tacotron-own-voice-training/wav/wav/'):
     if file.endswith(".wav"):
        path1='/kaggle/input/tacotron-own-voice-training/wav/wav/'+file
        path2='/kaggle/working/tacotron2/wav/'+file
        !sox $path1 -r 22050 -c 1 $path2

sox WARN rate: rate clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN rate: rate clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN rate: rate clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN rate: rate clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN rate: rate clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN rate: rate clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN rate: rate clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN rate: rate clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN rate: rate clipped 1 samples; decrease volume?
sox WARN dither:

In [9]:
!soxi /kaggle/working/tacotron2/wav/000.wav


Input File     : '/kaggle/working/tacotron2/wav/000.wav'
Channels       : 1
Sample Rate    : 22050
Precision      : 16-bit
Duration       : 00:00:01.54 = 34034 samples ~ 115.762 CDDA sectors
File Size      : 68.1k
Bit Rate       : 353k
Sample Encoding: 16-bit Signed Integer PCM



In [10]:
import json

os.chdir("/kaggle/working/hifi-gan")

with open('config_v1b.json') as f:
  json_config = json.load(f)
  json_config["batch_size"] = BATCH_SIZE_INP

with open('config_v1b.json', 'w') as JSON_FILE:
  json.dump(json_config, JSON_FILE)

In [11]:
import pandas as pd

metadata = pd.read_csv("/kaggle/input/tacotron-own-voice-training/metadata.csv")

total = len(metadata)
split = 0.85
train = int(total * split)

train_metadata = metadata[:train]
val_metadata = metadata[train:]

with open('/kaggle/working/tacotron2/filelists/ljs_audio_text_train_filelist.txt', 'w') as f:
    for _, (index, text, _) in train_metadata.iterrows():
        filepath = '/kaggle/working/tacotron2/wav/%03d.wav' % index
        f.write(filepath + '|' + text + '\n')

with open('/kaggle/working/tacotron2/filelists/ljs_audio_text_test_filelist.txt', 'w') as f:
    for _, (index, text, _) in val_metadata.iterrows():
        filepath = '/kaggle/working/tacotron2/wav/%03d.wav' % index
        f.write(filepath + '|' + text + '\n')
        

In [12]:
with open('/kaggle/working/tacotron2/filelists/ljs_audio_text_train_filelist.txt') as f:
    taco_filelist = f.readlines()
with open('/kaggle/working/tacotron2/filelists/ljs_audio_text_test_filelist.txt') as f:
     taco_filelist.extend(f.readlines())

In [15]:
import os
import shutil
os.chdir('/kaggle/working/tacotron2')
if os.path.exists("/kaggle/working/tacotron2/wavs"):
    shutil.rmtree("/kaggle/working/tacotron2/wavs")
os.mkdir("wavs")
os.chdir("wavs")
    
# os.system(f'cp -a /kaggle/input/tacotron-own-voice-training/wav ../')
    
if os.path.exists("/kaggle/working/tacotron2/wavs/wavs"):
    shutil.move("/kaggle/working/tacotron2/wavs/wavs", "/kaggle/working/tacotron2/tempwavs")
    shutil.rmtree("/kaggle/working/tacotron2/wavs")
    shutil.move("/kaggle/working/tacotron2/tempwavs", "/kaggle/working/tacotron2/wavs")

os.chdir('/kaggle/working/tacotron2')
shutil.copyfile(tacotron_model, "/kaggle/working/tacotron2/tacomodel")
seen_files = []
with open("/kaggle/working/tacotron2/filelists/ljs_audio_text_train_filelist.txt", "w") as f:
    for x in taco_filelist:
        if x.split("|")[0] not in seen_files:
            seen_files.append(x.split("|")[0])
            f.write(x)
if os.path.exists("/kaggle/working/tacotron2/GTAOutput"):
    shutil.rmtree("/kaggle/working/tacotron2/GTAOutput")
os.mkdir("GTAOutput")
!python GTA.py -c tacomodel -o GTAOutput

FP16 Run: False
Dynamic Loss Scaling: True
Distributed Run: False
cuDNN Enabled: True
cuDNN Benchmark: False
Warm starting model from checkpoint 'tacomodel'
100%|███████████████████████████████████████| 1000/1000 [06:49<00:00,  2.44it/s]


---
# **(CONTINUING TO TRAIN?)** 3.5 Transfer Model into the working directory

In [ ]:
CONTINUEQ = input("Are you continuing to train your model? [Y/N]: ").upper()
while CONTINUEQ not in ("Y","N"):
    CONTINUEQ = input("Please only enter Y or N\nAre you continuing to train your model? [Y/N]: ")

if CONTINUEQ == "Y":
    os.makedirs(f'../hifimodels/{CHARACTERNAME}')
    print("NOT THE MODEL FILENAME ITSELF, ONLY THE NAME YOU HAD GIVEN IT WHEN IMPORTING AS DATA INTO KAGGLE")
    MODELDATA_FILENAME = input("What is the name of the name you had assigned your HiFi-GAN model folder within your KAGGLE ACCOUNT?: ")
    os.system(f'cp -a ../../input/{MODELDATA_FILENAME}/{CHARACTERNAME} ../hifimodels/')
else:
    print("Ok")

---
# 4 Train HiFI-GAN for 5000+ steps (recommended) but if your tacotron 2 model sounds a little too rough you should train 
Stop the cell to finish the training

In [16]:
f = open('/kaggle/working/tacotron2/filelists/ljs_audio_text_train_filelist.txt', 'r')
content = f. read()
with open('training_file.txt', 'w') as f:
    f.write(content[:64499])

In [17]:
f = open('/kaggle/working/tacotron2/filelists/ljs_audio_text_train_filelist.txt', 'r')
content = f. read()
with open('validation_file.txt', 'w') as f:
    f.write(content[64499:])

In [18]:
train_filelist = f'/kaggle/working/tacotron2/training_file.txt'
val_filelist = f'/kaggle/working/tacotron2/validation_file.txt'

In [19]:
checkpoint_interval = 250

d = 'https://drive.google.com/uc?id='

os.chdir("/kaggle/working/hifi-gan")
if os.path.exists("/kaggle/working/hifi-gan/ft_dataset"):
    shutil.rmtree("/kaggle/working/hifi-gan/ft_dataset")
shutil.copytree("/kaggle/working/tacotron2/GTAOutput/mels/", "/kaggle/working/hifi-gan/ft_dataset/wavs/")
if os.path.exists("/kaggle/working/hifi-gan/wavs"):
    shutil.rmtree("/kaggle/working/hifi-gan/wavs")
shutil.copytree("/kaggle/working/tacotron2/wav/", "/kaggle/working/hifi-gan/wavs/")
shutil.copyfile(train_filelist, "/kaggle/working/hifi-gan/training.txt")
shutil.copyfile(val_filelist, "/kaggle/working/hifi-gan/validation.txt")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(os.path.join(output_dir, "do_00000000")):
    print("Downloading universal model...")
    os.system(f"gdown {d}1O63eHZR9t1haCdRHQcEgMfMNxiOciSru -O {os.path.join(output_dir, 'do_00000000')}")
    os.system(f"gdown {d}1qpgI41wNXFcH-iKq1Y42JlBC9j0je8PW -O {os.path.join(output_dir, 'g_00000000')}")
    start_from_universal = "--warm_start True "
else:
    start_from_universal = ""

!python train.py --fine_tuning True --config config_v1b.json \
{start_from_universal}\
--checkpoint_interval {checkpoint_interval} --checkpoint_path "{output_dir}" \
--input_training_file "/kaggle/working/hifi-gan/training.txt" \
--input_validation_file "/kaggle/working/hifi-gan/validation.txt" \
--input_wavs_dir "" --input_mels_dir "ft_dataset"

Downloading...
From (uriginal): https://drive.google.com/uc?id=1O63eHZR9t1haCdRHQcEgMfMNxiOciSru
From (redirected): https://drive.google.com/uc?id=1O63eHZR9t1haCdRHQcEgMfMNxiOciSru&confirm=t&uuid=71358c20-4274-49e9-a479-cbbed114e0b0
To: /kaggle/working/hifimodels/-/do_00000000
100%|██████████| 960M/960M [00:10<00:00, 89.9MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1qpgI41wNXFcH-iKq1Y42JlBC9j0je8PW
To: /kaggle/working/hifimodels/-/g_00000000
100%|██████████| 55.8M/55.8M [00:01<00:00, 46.7MB/s]


Initializing Training Process..
Batch size per GPU : 32
Generator(
  (conv_pre): Conv1d(80, 512, kernel_size=(7,), stride=(1,), padding=(3,))
  (ups): ModuleList(
    (0): ConvTranspose1d(512, 256, kernel_size=(16,), stride=(8,), padding=(4,))
    (1): ConvTranspose1d(256, 128, kernel_size=(16,), stride=(8,), padding=(4,))
    (2): ConvTranspose1d(128, 64, kernel_size=(4,), stride=(2,), padding=(1,))
    (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,), padding=(1,))
  )
  (resblocks): ModuleList(
    (0): ResBlock1(
      (convs1): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
        (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
      )
      (convs2): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,)

In [65]:
import os
st='/kaggle/working/tacotron2/wav/000.wav|Shut up 15 year-old'
# /kaggle/working/tacotron2/mel/499.npy
print(f"{st.split('wav')[0]}" + "mel"+f"{st.split('wav')[1]}"+'npy')
print(os.path.splitext(st)[0] + '.npy')

/kaggle/working/tacotron2/mel/000.npy
/kaggle/working/tacotron2/wav/000.npy


---
# 5 Zip up output folder and download (The ```g_00000000``` file is the file you'd need for synthesising)

Run the code cell below to zip up your files then **ONLY** click ```Download here``` once all files have been successfully zipped up

<a href="HiFi-GAN_Model.zip/"> Download here </a>

In [20]:
!zip -r ../HiFi-GAN_Model.zip ../hifimodels/

  adding: ../hifimodels/ (stored 0%)
  adding: ../hifimodels/-/ (stored 0%)
  adding: ../hifimodels/-/do_00000000 (deflated 6%)
  adding: ../hifimodels/-/config.json (deflated 48%)
  adding: ../hifimodels/-/g_00000000 (deflated 7%)
  adding: ../hifimodels/-/logs/ (stored 0%)
  adding: ../hifimodels/-/logs/events.out.tfevents.1690625910.6115fc56d6f0.1733.0 (deflated 66%)


In [40]:
!python inference.py --checkpoint_file /kaggle/working/hifimodels/-/g_00000000

Initializing Inference Process..
Loading '/kaggle/working/hifimodels/-/g_00000000'
Complete.
Removing weight norm...
/opt/conda/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/opt/conda/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  normalized, onesided, return_complex)
generated_files/020_generated.wav
generated_files/253_generated.wav
generated

In [33]:
pwd

'/kaggle/working/hifi-gan'

In [34]:
mkdir test_files

In [38]:
os.system(f'cp -a /kaggle/working/tacotron2/wav /kaggle/working/hifi-gan')

0

In [39]:
os.rename(f'/kaggle/working/hifi-gan/wav', f'/kaggle/working/hifi-gan/test_files')

In [44]:
os.rmdir(f'/kaggle/working/hifi-gan/test_mel_files')

In [45]:
os.system(f'cp -a /kaggle/working/tacotron2/mel /kaggle/working/hifi-gan')

0

In [46]:
os.rename(f'/kaggle/working/hifi-gan/mel', f'/kaggle/working/hifi-gan/test_mel_files')

In [50]:
shutil.rmtree(f'/kaggle/working/hifi-gan/generated_files')

In [51]:
!python inference_e2e.py --checkpoint_file /kaggle/working/hifimodels/-/g_00000000

Initializing Inference Process..
Loading '/kaggle/working/hifimodels/-/g_00000000'
Complete.
Removing weight norm...
generated_files_from_mel/354.wav
generated_files_from_mel/909.wav
generated_files_from_mel/197.wav
generated_files_from_mel/005.wav
generated_files_from_mel/139.wav
generated_files_from_mel/343.wav
generated_files_from_mel/549.wav
generated_files_from_mel/948.wav
generated_files_from_mel/892.wav
generated_files_from_mel/910.wav
generated_files_from_mel/902.wav
generated_files_from_mel/629.wav
generated_files_from_mel/022.wav
generated_files_from_mel/278.wav
generated_files_from_mel/989.wav
generated_files_from_mel/923.wav
generated_files_from_mel/793.wav
generated_files_from_mel/357.wav
generated_files_from_mel/322.wav
generated_files_from_mel/054.wav
generated_files_from_mel/952.wav
generated_files_from_mel/920.wav
generated_files_from_mel/560.wav
generated_files_from_mel/439.wav
generated_files_from_mel/308.wav
generated_files_from_mel/445.wav
generated_files_from_mel/